# Test 1

In [1]:
import numpy as np
import robustsp as rsp
from scipy.stats import chi2
import math
   
yx = np.asarray([1.3,4,5])

Xx = np.asarray([[4,5,6],[7,8,9],[0,3,0]])
c = 4
lambd = 3.3
b0 = np.zeros(3)
sig0 = 1
reltol = 1e-5
printitn = 0
iterMAX = 500

rsp.hublasso(yx,Xx,lambd,b0,sig0,c)

c:\users\computer\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\scimath.py:461: RuntimeWarning: overflow encountered in power
  return nx.power(x, p)
c:\users\computer\desktop\robustsp\robustsp\Regression\hublasso.py:73: RuntimeWarning: overflow encountered in matmul
  betaold@b0))\


(array([-3.25655076e+302,  4.73680110e+302, -4.44075103e+302]),
 9.251453164795978e+300,
 array([-3.70058127e+301, -3.70058127e+301, -3.70058127e+301]))

* b0 =

  1.0e+302 *

   -3.2566
    4.7368
   -4.4408

* sig0 =

  9.2515e+300

* psires =

  1.0e+301 *

   -3.7006
   -3.7006
   -3.7006

# Test 2

In [12]:
# Sig output wrong

import numpy as np
import robustsp as rsp
from scipy.stats import chi2
import math

yx = np.array([ 1.375, -2.125, -0.125, 0.875])

Xx = np.array([[ 0.8660254 , -0.28867513, -0.28867513, -0.28867513],
  [-0.28867513,  0.8660254 , -0.28867513, -0.28867513],
  [-0.28867513, -0.28867513,  0.8660254 , -0.28867513],
  [-0.28867513, -0.28867513, -0.28867513,  0.8660254 ]])

lambd = 0.6163512806474756

b0 = np.array([0., 0., 0., 0.])

sig0 = 1.812638977729619

c = 1.3415

reltol = 1e-5

printitn = 0
iterMAX = 500
print(rsp.hublasso(yx,Xx,lambd,b0,sig0,c))
###################################################################

# ensure that y is Nx1 and not just N and proper formats
y = np.copy(np.asarray(yx))
X = np.copy(np.asarray(Xx))
y = y if not len(y.shape)==2 else y.flatten()

n,p = X.shape
realdata = np.isrealobj(y)

if c is None: c = 1.3415 if realdata else 1.215
# Default: approx 95 efficiency for Gaussian errors

csq = c**2

if realdata:
    qn = chi2.cdf(csq,1)
    alpha = chi2.cdf(csq,3)+csq*(1-qn) # consistency factor for scale
else:
    qn = chi2.cdf(2*csq,2)
    alpha = chi2.cdf(2*csq,4)+csq*(1-qn) # consistency factor for scale

con = np.sqrt(n*alpha)
normb0 = np.linalg.norm(b0)
b0 = b0.astype(np.longdouble)
betaold = np.copy(b0)

i = 0
for _ in range(iterMAX):
    r = y-X@b0[:,np.newaxis].flatten()

    psires = rsp.psihub(r/sig0,c)*sig0
    sig1 = np.linalg.norm(psires)/con
    if sig1 == np.Inf:
        print(r)
        print(psires)
        print(con)
    crit2 = np.abs(sig0-sig1)
    
    for jj in range(p):
        psires = rsp.psihub(r/sig1,c)*sig1 # Update the pseudo-residual
        if any(map(lambda x: math.isnan(x), psires)):
            print(r)
            print(sig1)
        b0[jj] = rsp.SoftThresh(b0[jj]+X[:,jj].T @ psires,lambd)
        r+=X[:,jj]*(betaold[jj]-b0[jj])


    normb = np.linalg.norm(b0)
    crit = np.sqrt(normb0**2 + normb**2 -2*np.real(betaold@b0))/normb

    if printitn > 0:
        pass
        '''
        r = (y-X@b0)/sig1
        objnew = (sig1)*np.sum(rsp.rhofun(r,c)+())
        '''
    if crit<reltol: break

    sig0 = sig1
    betaold = np.copy(b0)
    normb0 = normb

    i+=1
    
if printitn > 0:
    b0[np.abs(b0)<5e-9]=0
    r = y- X@b0
    s = np.sign(b0)
    ind = np.arange(p)
    ind2 = ind[s==0]
    psires = rsp.psihub(r/sig1,c)*sig1
    s[ind2] = X[:,ind2]@psires/lambd
    fpeq = -X.T@psires + lambd*s # FP equation equal to zero
    print('lam = %.4f it = %d norm(FPeq1)= %.12f abs(FPeq2)=%.12f\n' \
    % (lambd,i, np.linalg.norm(fpeq),sig1-np.linalg.norm(psires)/con))


(array([ 0.4037636 , -1.70279951, -0.        ,  0.        ]), 0.6143870137430529, array([ 0.5337746 , -0.53377587, -0.49999936,  0.50000064]), 8)


0.6143870137430529

In [10]:
rsp.hublasso(yx,Xx,lambd,b0,sig0,c)

(array([ 0.40376213, -1.7028    , -0.        ,  0.        ]),
 0.6143870137430529,
 array([ 0.53377573, -0.53377587, -0.49999993,  0.50000007]),
 1)

In [31]:
1.e500*np.ones((3,3))@np.ones((3,1))*1e100 # all inf
np.linalg.norm(1.e500*np.ones((3,3),dtype=np.longdouble)@np.ones((3,1),dtype=np.longdouble)*1e100)

inf

In [18]:
-1.0257e04+np.asarray([[6],[9],[0]]).T \
@ np.asarray([3.4326, 3.4326, 3.4326])*1.0e03

array([41232.])

In [43]:
sig1 = np.linalg.norm([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])/1.7319464444221777 # inf
psihub = lambda x,c: (x * (np.abs(x)<=c) + c*np.sign(x) *(np.abs(x)>c))
psihub(np.array([-4.92776165e+155, -7.66540701e+155,  4.38023258e+155])/sig1,c)*sig1
np.Inf/np.Inf
sig1

c:\users\computer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until


inf

In [53]:
np.sqrt(np.sum(np.array([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])**2))/np.sqrt(3)
# expected result: 1.14067182e+154
# what I get: inf

np.sqrt(np.sum(np.array([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154],dtype=np.longdouble)**2))/np.sqrt(3)

'''
norm([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])

ans =

  1.9757e+154
'''
np.linalg.norm([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154]) # inf
np.array([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])**2 # works
np.sum(np.array([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])**2) # inf
np.sum(np.abs([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])**2) # inf, using absolute
# changing dtype to longdouble
np.linalg.norm(np.array([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154],dtype=np.longdouble)) # inf
import scipy
scipy.linalg.norm([-1.14067182e+154, -1.14067182e+154 , 1.14067182e+154])

1.975701547002061e+154

In [2]:
import scipy
scipy.linalg.norm([-3.70058127e+301, -3.70058127e+301, -3.70058127e+301])

6.409594777177762e+301